In [1]:
import PIL
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from rembg import remove

PIL.Image.MAX_IMAGE_PIXELS = None #Increase image size limit

In [2]:
def detect_leaves(input_image):
    """
    Detects and returns bounding boxes around specific areas (leaves) in the input image.
    
    Args:
    input_image (numpy.ndarray): Input image to detect leaves from.
    
    Returns:
    numpy.ndarray: Array containing bounding boxes around leaves.
    """

    # Appliquer un gros flou sur l'image
    image_floue = cv2.blur(input_image, (80, 80))  # Vous pouvez ajuster la taille du noyau pour contrôler l'intensité du flou

    # Threshold area to filter small contours
    THRESHOLD_AREA = 600000

    # Binariser l'image floue en utilisant un seuil spécifique
    seuil = 128  # Seuil de binarisation (modifier selon vos besoins)
    valeur_maximale = 255  # Valeur maximale pour les pixels binarisés
    _, image_floue_binarisee = cv2.threshold(image_floue, seuil, valeur_maximale, cv2.THRESH_BINARY)
    image_floue_binarisee = cv2.cvtColor(image_floue_binarisee, cv2.COLOR_BGR2GRAY)

    # Binariser l'image en utilisant un seuil
    _, binary_image = cv2.threshold(image_floue_binarisee, 245, 255, cv2.THRESH_BINARY_INV)

    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Initialize list to store detected bounding boxes
    bounding_boxes = []

    for c in contours:
        area = cv2.contourArea(c)
        if area > THRESHOLD_AREA:  # Adjust threshold_area to your desired value
            x, y, w, h = cv2.boundingRect(c)
            if w > 400 and h > 5000:
                bounding_boxes.append([x, y, x+w, y+h])

    # Convert list of bounding boxes to numpy array and return
    return np.array(bounding_boxes)

In [3]:
def is_image_unusable(image):
    """
    Checks if the image is unusable based on its dimensions.
    
    Args:
    image (numpy.ndarray): Input image to check for usability.
    
    Returns:
    bool: True if the image is unusable, False otherwise.
    """
    # Get the height and width of the image
    height = image.shape[0]
    
    # Define the threshold values for image dimensions as constants
    MIN_HEIGHT = 11000
    MAX_HEIGHT = 22500
    
    # Check if the height is outside the acceptable range
    if height < MIN_HEIGHT or height > MAX_HEIGHT:
        return True
    else:
        return False

In [14]:
import os
import cv2

def image_processing(entry_path, exit_path):
    """
    Process images in the specified directory.
    
    Args:
    entry_path (str): Path to the directory containing input images.
    exit_path (str): Path to the directory where processed images will be saved.
    """
    # Initialize counters
    num_files = 0
    count_files = 0
    count_unusable_files = 0
    
    # Count the number of JPEG files in the entry directory
    for filename in os.listdir(entry_path):
        if filename.lower().endswith('.jpg'):
            num_files += 1

    # Create the output directory 'new_results'
    new_results_path = os.path.join(exit_path, "new_results")
    exit_path = new_results_path
    os.makedirs(new_results_path, exist_ok=True)  # This line creates the directory if it doesn't exist
    
    # Create subdirectories 'File' and 'Unusable_File' inside 'new_results'
    file_path = os.path.join(new_results_path, "File")
    unusable_file_path = os.path.join(new_results_path, "Unusable_File")
    os.makedirs(file_path, exist_ok=True)
    os.makedirs(unusable_file_path, exist_ok=True)

    # Process each JPEG file in the entry directory
    for index, filename in enumerate(os.listdir(entry_path)):
        full_path = os.path.join(entry_path, filename)
        
        # Check if the file is a JPEG image
        if filename.lower().endswith('.jpg'):
            # Read the image
            img = cv2.imread(full_path)
            count_files += 1

            # Check if the image is unusable
            if is_image_unusable(img):
                count_unusable_files += 1
                cv2.imwrite(os.path.join(unusable_file_path, f"Unusable_File_{filename}"), img)
                continue
            else:
                # Detect leaves and save each leaf as a separate image
                bounding_boxes = detect_leaves(img)
                file_name_without_extension, extension = os.path.splitext(filename)
                for j, box in enumerate(bounding_boxes):
                    x1, y1, x2, y2 = box
                    part = img[y1:y2, x1:x2]

                    # Construct the new file name with leaf index and reattach the extension
                    new_file_name = f"{file_name_without_extension}_leaf{j + 1}{extension}"
                    cv2.imwrite(os.path.join(file_path, new_file_name), part)

        print(f"Usable Files: {count_files - count_unusable_files}, Unusable Files: {count_unusable_files}")


In [19]:
image_processing("/Users/titouanlegourrierec/Desktop/all_scan_data/all_pepi_290323", "/Users/titouanlegourrierec/Desktop")

Corrupt JPEG data: 3769 extraneous bytes before marker 0xd2
Corrupt JPEG data: 4579 extraneous bytes before marker 0xd3


Usable Files: 1, Unusable Files: 1


Corrupt JPEG data: 4645 extraneous bytes before marker 0xd7


Usable Files: 2, Unusable Files: 1


Corrupt JPEG data: 5038 extraneous bytes before marker 0xd0


...

Usable Files: 180, Unusable Files: 21


Corrupt JPEG data: 5030 extraneous bytes before marker 0xd3


Usable Files: 181, Unusable Files: 21


Corrupt JPEG data: 3434 extraneous bytes before marker 0xd6
Corrupt JPEG data: 4595 extraneous bytes before marker 0xd4


Usable Files: 182, Unusable Files: 22


---
# Brouillon

In [10]:
import os
import cv2
import tkinter as tk
from tkinter import ttk
import time

def format_time(seconds):
    """
    Format the time in seconds into a string of minutes and seconds.
    
    Args:
    seconds (float): Time in seconds.
    
    Returns:
    str: Formatted time string in the format "X min Y s".
    """
    minutes = int(seconds // 60)
    seconds %= 60
    return f"{minutes} min {seconds:.0f} s"

def image_processing(entry_path, exit_path, progress_bar, label_status):
    """
    Process images in the specified directory.
    
    Args:
    entry_path (str): Path to the directory containing input images.
    exit_path (str): Path to the directory where processed images will be saved.
    progress_bar (ttk.Progressbar): Progress bar widget.
    label_status (tk.Label): Label widget to display status messages.
    """
    # Initialize counters
    num_files = 0
    count_files = 0
    count_unusable_files = 0
    total_time = 0
    
    # Count the number of JPEG files in the entry directory
    for filename in os.listdir(entry_path):
        if filename.lower().endswith('.jpg'):
            num_files += 1

    # Create the output directory 'new_results'
    new_results_path = os.path.join(exit_path, "new_results")
    exit_path = new_results_path
    os.makedirs(new_results_path, exist_ok=True)  # This line creates the directory if it doesn't exist
    
    # Create subdirectories 'File' and 'Unusable_File' inside 'new_results'
    file_path = os.path.join(new_results_path, "File")
    unusable_file_path = os.path.join(new_results_path, "Unusable_File")
    os.makedirs(file_path, exist_ok=True)
    os.makedirs(unusable_file_path, exist_ok=True)

    # Set up the progress bar
    progress_bar['maximum'] = num_files

    start_time = time.time()  # Record start time

    # Process each JPEG file in the entry directory
    for index, filename in enumerate(os.listdir(entry_path)):
        full_path = os.path.join(entry_path, filename)
        
        # Check if the file is a JPEG image
        if filename.lower().endswith('.jpg'):
            # Read the image
            img = cv2.imread(full_path)
            count_files += 1

            # Update status label
            label_status.config(text=f"Processing file {count_files}/{num_files}")

            # Check if the image is unusable
            if is_image_unusable(img):
                count_unusable_files += 1
                cv2.imwrite(os.path.join(unusable_file_path, f"Unusable_File_{filename}"), img)
                progress_bar.step(1)
                window.update()  # Force update of the GUI
                continue
            else:
                # Detect leaves and save each leaf as a separate image
                start_file_time = time.time()  # Record start time for the current file
                bounding_boxes = detect_leaves(img)
                file_name_without_extension, extension = os.path.splitext(filename)
                for j, box in enumerate(bounding_boxes):
                    x1, y1, x2, y2 = box
                    part = img[y1:y2, x1:x2]
                    
                    # Construct the new file name with leaf index and reattach the extension
                    new_file_name = f"{file_name_without_extension}_leaf{j + 1}{extension}"
                    cv2.imwrite(os.path.join(file_path, new_file_name), part)
                
                # Update progress bar
                progress_bar.step(1)
                window.update()  # Force update of the GUI

                # Calculate elapsed time for the current file
                file_elapsed_time = time.time() - start_file_time
                total_time += file_elapsed_time

                # Calculate average time per file until now
                average_time_per_file = total_time / count_files

                # Estimate remaining time
                remaining_time = (num_files - count_files) * average_time_per_file

                # Update status label with estimated time remaining
                label_status.config(text=f"Processing file {count_files}/{num_files} - Est. time remaining: {format_time(remaining_time)}")
                label_status.update()  # Force update of the label

    print(f"Usable Files: {count_files - count_unusable_files}, Unusable Files: {count_unusable_files}")

# Example usage:
def start_processing():
    entry_path = "/Users/titouanlegourrierec/Desktop/all_scan_data/all_090523"
    exit_path = "/Users/titouanlegourrierec/Desktop"
    image_processing(entry_path, exit_path, progress_bar, label_status)

# Create tkinter window
window = tk.Tk()
window.title("Image Processing")
window.geometry("400x150")

# Create progress bar
progress_bar = ttk.Progressbar(window, orient=tk.HORIZONTAL, length=300, mode='determinate')
progress_bar.pack(pady=10)

# Create label for status messages
label_status = tk.Label(window, text="")
label_status.pack()

# Button to start processing
start_button = tk.Button(window, text="Start Processing", command=start_processing)
start_button.pack()

window.mainloop()


In [4]:
import os
import cv2
import tkinter as tk
from tkinter import ttk, filedialog
import time

def format_time(seconds):
    """
    Format the time in seconds into a string of minutes and seconds.
    
    Args:
    seconds (float): Time in seconds.
    
    Returns:
    str: Formatted time string in the format "X min Y s".
    """
    minutes = int(seconds // 60)
    seconds %= 60
    return f"{minutes} min {seconds:.0f} s"

def image_processing(entry_path, exit_path, progress_bar, label_status):
    """
    Process images in the specified directory.
    
    Args:
    entry_path (str): Path to the directory containing input images.
    exit_path (str): Path to the directory where processed images will be saved.
    progress_bar (ttk.Progressbar): Progress bar widget.
    label_status (tk.Label): Label widget to display status messages.
    """
    # Initialize counters
    num_files = 0
    count_files = 0
    count_unusable_files = 0
    total_time = 0
    
    # Count the number of JPEG files in the entry directory
    for filename in os.listdir(entry_path):
        if filename.lower().endswith('.jpg'):
            num_files += 1

    # Create the output directory 'new_results'
    new_results_path = os.path.join(exit_path, "new_results")
    exit_path = new_results_path
    os.makedirs(new_results_path, exist_ok=True)  # This line creates the directory if it doesn't exist
    
    # Create subdirectories 'File' and 'Unusable_File' inside 'new_results'
    file_path = os.path.join(new_results_path, "File")
    unusable_file_path = os.path.join(new_results_path, "Unusable_File")
    os.makedirs(file_path, exist_ok=True)
    os.makedirs(unusable_file_path, exist_ok=True)

    # Set up the progress bar
    progress_bar['maximum'] = num_files

    start_time = time.time()  # Record start time

    # Process each JPEG file in the entry directory
    for index, filename in enumerate(os.listdir(entry_path)):
        full_path = os.path.join(entry_path, filename)
        
        # Check if the file is a JPEG image
        if filename.lower().endswith('.jpg'):
            # Read the image
            img = cv2.imread(full_path)
            count_files += 1

            # Update status label
            label_status.config(text=f"Processing file {count_files}/{num_files}")

            # Check if the image is unusable
            if is_image_unusable(img):
                count_unusable_files += 1
                cv2.imwrite(os.path.join(unusable_file_path, f"Unusable_File_{filename}"), img)
                progress_bar.step(1)
                window.update()  # Force update of the GUI
                continue
            else:
                # Detect leaves and save each leaf as a separate image
                start_file_time = time.time()  # Record start time for the current file
                bounding_boxes = detect_leaves(img)
                file_name_without_extension, extension = os.path.splitext(filename)
                for j, box in enumerate(bounding_boxes):
                    x1, y1, x2, y2 = box
                    part = img[y1:y2, x1:x2]
                    
                    # Construct the new file name with leaf index and reattach the extension
                    new_file_name = f"{file_name_without_extension}_leaf{j + 1}{extension}"
                    cv2.imwrite(os.path.join(file_path, new_file_name), part)
                
                # Update progress bar
                progress_bar.step(1)
                window.update()  # Force update of the GUI

                # Calculate elapsed time for the current file
                file_elapsed_time = time.time() - start_file_time
                total_time += file_elapsed_time

                # Calculate average time per file until now
                average_time_per_file = total_time / count_files

                # Estimate remaining time
                remaining_time = (num_files - count_files) * average_time_per_file

                # Update status label with estimated time remaining
                label_status.config(text=f"Processing file {count_files}/{num_files} - Est. time remaining: {format_time(remaining_time)}")
                label_status.update()  # Force update of the label

    print(f"Usable Files: {count_files - count_unusable_files}, Unusable Files: {count_unusable_files}")

def select_entry_path():
    entry_path = filedialog.askdirectory()
    entry_path_var.set(entry_path)

def select_exit_path():
    exit_path = filedialog.askdirectory()
    exit_path_var.set(exit_path)

# Create tkinter window
window = tk.Tk()
window.title("Image Processing")
window.geometry("400x200")

# Create entry and exit path variables
entry_path_var = tk.StringVar()
exit_path_var = tk.StringVar()

# Create entry path selection button and label
entry_path_button = tk.Button(window, text="Select Entry Path", command=select_entry_path)
entry_path_button.pack(pady=5)
entry_path_label = tk.Label(window, textvariable=entry_path_var)
entry_path_label.pack(pady=5)

# Create exit path selection button and label
exit_path_button = tk.Button(window, text="Select Exit Path", command=select_exit_path)
exit_path_button.pack(pady=5)
exit_path_label = tk.Label(window, textvariable=exit_path_var)
exit_path_label.pack(pady=5)

# Create progress bar
progress_bar = ttk.Progressbar(window, orient=tk.HORIZONTAL, length=300, mode='determinate')
progress_bar.pack(pady=10)

# Create label for status messages
label_status = tk.Label(window, text="")
label_status.pack()

# Button to start processing
start_button = tk.Button(window, text="Start Processing", command=lambda: image_processing(entry_path_var.get(), exit_path_var.get(), progress_bar, label_status))
start_button.pack()

window.mainloop()


Corrupt JPEG data: 3769 extraneous bytes before marker 0xd2
Corrupt JPEG data: 4579 extraneous bytes before marker 0xd3
Corrupt JPEG data: 4645 extraneous bytes before marker 0xd7
Corrupt JPEG data: 5038 extraneous bytes before marker 0xd0
Corrupt JPEG data: 4960 extraneous bytes before marker 0xd3
Corrupt JPEG data: 4602 extraneous bytes before marker 0xd7
Corrupt JPEG data: 4357 extraneous bytes before marker 0xd3
Corrupt JPEG data: 4514 extraneous bytes before marker 0xd7
Corrupt JPEG data: 4794 extraneous bytes before marker 0xd1
Corrupt JPEG data: 4823 extraneous bytes before marker 0xd7
Corrupt JPEG data: 5064 extraneous bytes before marker 0xd6
Corrupt JPEG data: 4476 extraneous bytes before marker 0xd3
Corrupt JPEG data: 4743 extraneous bytes before marker 0xd1
Corrupt JPEG data: 4499 extraneous bytes before marker 0xd1
Corrupt JPEG data: 4769 extraneous bytes before marker 0xd4
Corrupt JPEG data: 4573 extraneous bytes before marker 0xd2
Corrupt JPEG data: 4519 extraneous bytes